<a href="https://colab.research.google.com/github/yecatstevir/teambrainiac/blob/main/source/DL/Group_3DCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning with PyTorch
## 3D Convolutional Neural Network on Group Brain fMRI
Contributors: Stacey Rivet Beck, Ben Merrill
### To Do:
- Either:
  - Get Raw data in 4D   
          - OR -
  - Reshape Raw data into 4D from 2D
    - Apply Whole Brain Mask to data and save to AWS

- Build Dataloader: https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

- Implement 3DCNN from paper: REALLY GREAT PAPERS
  - Nguyen et al. http://proceedings.mlr.press/v136/nguyen20a/nguyen20a.pdf
  - Wang et al. https://arxiv.org/pdf/1801.09858.pdf (discusses more in detail the input data shapes and processing)
  
  - Inputs: 84@ x * y * z ; one fmri time series at a time, not concatenated
  - Basic Architecture:
        #First layer is generating temporal descriptors of the voxels
        Conv1 1 x 1 x 1 filter, output = 32, stride = 1, ReLU, BatchNorm
        Conv2 7 x 7 x 7 filter, output = 64, stride = 2, ReLU, BatchNorm
        Conv3 3 x 3 x 3 fitler, output = 64, stride = 2, ReLU, BatchNorm
        Conv4 3 x 3 x 3 fitler, output = 128, stride = 2, ReLU, BatchNorm
        Global Average Pooling on final feature maps ->
        Flattened maps size 128?
        Fully connected layer size 64
        Fully connected layer size 2 (2 way classification, one for each class) -> softmax

        Optimized with Adam, standard parameters (β1=0.9 and β2=0.999)
        Batched at 32, but we may need to batch smaller due to GPU compute
        Learning Rate = 0.001, gradual decay after Val loss plateaued after 15 epochs
        Cross entropy Loss
        Employ early stopping
        In Wang et al. they used data for visualization, same size as input data, though are reduced in time dimension to be mapped on fsaverage surface. 
        
  

## Importing Dataset and Labels

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')  

Mounted at /content/gdrive


In [2]:
# Clone the entire repo.
!git clone -l -s https://github.com/yecatstevir/teambrainiac.git

# Change directory into cloned repo DL folder
%cd teambrainiac/source/DL

# !ls

Cloning into 'teambrainiac'...
remote: Enumerating objects: 1109, done.
remote: Counting objects: 100% (1109/1109), done.
remote: Compressing objects: 100% (857/857), done.
remote: Total 1109 (delta 701), reused 474 (delta 236), pack-reused 0
Receiving objects: 100% (1109/1109), 82.77 MiB | 28.76 MiB/s, done.
Resolving deltas: 100% (701/701), done.
/content/teambrainiac/source/DL


### Load path_config.py

In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving path_config.py to path_config.py
User uploaded file "path_config.py" with length 196 bytes


## Import Packages

In [4]:
# Possible Missing Packages
!pip install boto3

     |████████████████████████████████| 132 kB 25.5 MB/s 
     |████████████████████████████████| 79 kB 6.4 MB/s 
     |████████████████████████████████| 8.7 MB 70.8 MB/s 
     |████████████████████████████████| 138 kB 73.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [5]:
# General Library Imports
import re
import scipy.io
import os
import pickle
import numpy as np
import nibabel as nib
import pandas as pd
import boto3
import tempfile
import tqdm
import random
from path_config import mat_path
from botocore.exceptions import ClientError
from collections import defaultdict
from sklearn.preprocessing import StandardScaler

# From Local Directory
from access_data_dl import *
from process_dl import *

# Pytroch Libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

#import torchvision.transforms as transforms
from torch.nn import ReLU, CrossEntropyLoss, Conv3d, Module, Softmax, AdaptiveAvgPool3d
from torch.optim import Adam, SGD

#from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
# from dataloader_class import DatasetFmri


## Import Group fMRI Data, Normalize, and Create Masks

In [6]:
# Open path dictionary file to get subject ids
path = "../data/data_path_dictionary.pkl"
data_path_dict = open_pickle(path)

In [7]:
# Fix hyperparameters for preprocessing
label_type='rt_labels'
n_subjects = len(data_path_dict['subject_ID'])
runs_list = [2,3]


# Run functions to import unmasked data
image_label_mask, image_labels = labels_mask_binary(data_path_dict, label_type='rt_labels')

all_subjects = load_subjects_chronologically(data_path_dict, n_subjects, image_label_mask, image_labels, label_type, runs_list)

Subject ids loaded.
Adding subjects to dictionary.


52it [11:35, 13.37s/it]


In [8]:
def mask_normalize_runs_reshape_4d(chron_subject_dict, mask, scaler):
    """
    chron_subject_dict : (subject data returned from load_subjects_chronologically function, keys are subject IDs)
    mask               : The mask meant to be applied to each image, typically a whole brain mask
    scaler             : Scaler from sklearn used to normalize the data
    
    returns            : dictionary with fully normalized subjects and labels
                         each subject's run is in 5d - n_images x n_color_layers(i.e. 1 because images are black and white) x length x width x height
    """

    runs_normalized_subjects = {}
    for i,sub_id in enumerate(chron_subject_dict.keys()):
      temp_subject = {}
      for key in chron_subject_dict[sub_id].keys():
        if key == 'image_labels':
          # Labels for the images
          temp_subject['image_labels'] = chron_subject_dict[sub_id]['image_labels']
        
        else:
          # Subject Runs
          temp_run_unmasked = []
          temp_run = chron_subject_dict[sub_id][key]

          if scaler == 'standard':
            temp_scaler = StandardScaler()
          else:
            print('Please import required scaler and update function')
            break
          
          temp_run_masked = temp_run[:,mask]
          temp_run_masked_norm =  temp_scaler.fit_transform(temp_run_masked)

          temp_run_masked_norm_index = 0
          for t_or_f in mask:
            if t_or_f == True:
              temp_run_unmasked.append(temp_run_masked_norm[:,temp_run_masked_norm_index])
              temp_run_masked_norm_index += 1
            elif t_or_f == False:
              temp_run_unmasked.append(np.zeros(len(temp_run_masked_norm[:,0])))
            else:
              print('Error')
              break
        
          temp_run_unmasked = np.array(temp_run_unmasked).T
          temp_run_unmasked_4d = []
          for image in temp_run_unmasked:
            image_3d = image.reshape((79,95,79), order='F')
            image_4d = np.array([image_3d])
            temp_run_unmasked_4d.append(image_4d)
          temp_run_unmasked_4d = np.array(temp_run_unmasked_4d)
          temp_subject[key] = temp_run_unmasked_4d

      runs_normalized_subjects[sub_id] = temp_subject
      chron_subject_dict[sub_id] = None
      
      print('Completed Subject', str(i+1))

    return runs_normalized_subjects

In [9]:
# Get Mask Indicies. Note that 'mask' is the string for a whole brain mask.
whole_brain_mask = get_mask('mask', data_path_dict, mask_ind=0)
scaler = 'standard'

all_subjects_inital_reshape = mask_normalize_runs_reshape_4d(all_subjects, whole_brain_mask, scaler)

Completed Subject 1
Completed Subject 2
Completed Subject 3
Completed Subject 4
Completed Subject 5
Completed Subject 6
Completed Subject 7
Completed Subject 8
Completed Subject 9
Completed Subject 10
Completed Subject 11
Completed Subject 12
Completed Subject 13
Completed Subject 14
Completed Subject 15
Completed Subject 16
Completed Subject 17
Completed Subject 18
Completed Subject 19
Completed Subject 20
Completed Subject 21
Completed Subject 22
Completed Subject 23
Completed Subject 24
Completed Subject 25
Completed Subject 26
Completed Subject 27
Completed Subject 28
Completed Subject 29
Completed Subject 30
Completed Subject 31
Completed Subject 32
Completed Subject 33
Completed Subject 34
Completed Subject 35
Completed Subject 36
Completed Subject 37
Completed Subject 38
Completed Subject 39
Completed Subject 40
Completed Subject 41
Completed Subject 42
Completed Subject 43
Completed Subject 44
Completed Subject 45
Completed Subject 46
Completed Subject 47
Completed Subject 48
C

In [10]:
# Make sure that we have all subjects
len(all_subjects_inital_reshape.keys())

52

In [11]:
# Free up RAM by deleting some variables that are taking up memory
all_subjects = None

## Format Train and Test Data for Single Subject

In [20]:
'''
This is to keep hardcoded dictionary of shuffled ids to make sure train, validation, and test sets stay seperate

If the size you are looking for is not here, use the function below

five_ids = {'test': ['10017_08894'],
 'train': ['10008_09924', '10016_09694', '10004_08693'],
 'val': ['10009_08848']}


all_ids = {'test': ['10023_09126','30036_09758','30053_10112','10004_08693','30025_09402','10022_08854','10065_09587','30011_09170',
'30009_09227','10037_09903','10057_10124'],
 'train': ['30026_09430','30008_08981','10046_09216','30027_09638','10042_08990','10017_08894','30020_09236',
  '30017_09567','10039_08941','10035_08847','10069_09785','10027_09455','30004_08965','10053_09018','10043_09222',
  '10061_09308','30044_10095','10038_09063','10045_08968','10016_09694','30024_09398','10033_08871','10047_09030',
  '30035_09836','30012_09102','10018_08907','30045_10182','10034_08879','10009_08848','30033_09776',
  '30014_09352','10056_09615','10050_09079','10084_10188','10080_09931','10021_08839'],
 'val': ['10060_09359','30038_09967','10008_09924','10066_09687','10036_09800']}

'''
print()
# generate_train_val_test_dict(list(all_subjects_inital_reshape.keys()), train_val_test_proportion=[0.7,0.8,1])

In [22]:
all_ids = {'test': ['10023_09126','30036_09758','30053_10112','10004_08693','30025_09402','10022_08854','10065_09587','30011_09170',
'30009_09227','10037_09903','10057_10124'],
 'train': ['30026_09430','30008_08981','10046_09216','30027_09638','10042_08990','10017_08894','30020_09236',
  '30017_09567','10039_08941','10035_08847','10069_09785','10027_09455','30004_08965','10053_09018','10043_09222',
  '10061_09308','30044_10095','10038_09063','10045_08968','10016_09694','30024_09398','10033_08871','10047_09030',
  '30035_09836','30012_09102','10018_08907','30045_10182','10034_08879','10009_08848','30033_09776',
  '30014_09352','10056_09615','10050_09079','10084_10188','10080_09931','10021_08839'],
 'val': ['10060_09359','30038_09967','10008_09924','10066_09687','10036_09800']}

In [25]:
def train_test_aggregation_group(subjects_dict, runs, train_val_test_ids):
  """
    This function aggregates a group of subjects' runs into a training and test set
    split up by the desired proportion
    subjects_dict       : A dictionary of subject images with ids as keys 
    runs                : List of Runs to use from subject dict
    train_val_test_dict : dictionary of subject ids for train, validation, and test 
    
    returns            : train and test images and their labels ready for dataloader or to save on AWS s3
  """
  train_val_test_dict = {}

  for key in train_val_test_ids.keys():
    train_or_val_or_test = {}
    images = []
    labels = []
    temp_ids = train_val_test_ids[key]
    for subject_id in temp_ids:
      for run in runs:
        run_key = 'run_'+str(run)
        subject_images = subjects_dict[subject_id][run_key]
        for image in subject_images:
          images.append(image)
        labels.extend(list(subjects_dict[subject_id]['image_labels']))
      train_val_test_ids[subject_id] = None
    
    train_or_val_or_test['images'] = torch.from_numpy(np.array(images).astype(float))
    train_or_val_or_test['labels'] = torch.from_numpy(np.array(labels).astype(float)).long()
    train_val_test_dict[key] = train_or_val_or_test

  return train_val_test_dict

In [ ]:
runs = [2,3]

train_val_test_dict = train_test_aggregation_group(all_subjects_inital_reshape, runs, all_ids)

all_subjects_initial_reshape = None

In [110]:
for key in train_val_test_dict.keys():
  print(train_val_test_dict[key]['images'].shape)
  print(train_val_test_dict[key]['labels'].shape,'\n')


torch.Size([168, 1, 79, 95, 79])
torch.Size([168]) 

torch.Size([504, 1, 79, 95, 79])
torch.Size([504]) 

torch.Size([168, 1, 79, 95, 79])
torch.Size([168]) 



## Upload Data

In [ ]:
s3_upload(train_val_test_dict, 'dl/train_val_test_70_10_20.pkl', 'pickle')

## Build Dataloader

In [ ]:
def get_dataloader(train_dict, test_dict, bs):
  '''
  train_dict : Dictionary of training images and their labels
   
  test_dict  : Dictionary of testing images and their labels
  '''

    return (
        DataLoader(train_ds, batch_size=bs),#, shuffle=True),
        DataLoader(test_ds, batch_size=bs * 2)
    )

In [46]:
from torch.utils.data import TensorDataset

x_train = train_dict['images']
y_train = train_dict['labels']

x_test = test_dict['images']
y_test = test_dict['labels']

train_ds = TensorDataset(x_train, y_train)
test_ds = TensorDataset(x_test, y_test)

In [47]:
bs = 6

train_dl = DataLoader(train_ds, batch_size=bs)
test_dl = DataLoader(test_ds, batch_size=bs*2)

In [49]:
# bs = 6

# train_dl, test_dl = get_dataloader(train_ds, test_ds, bs)

In [50]:
# Sanity Check
train_features, train_labels = next(iter(train_dl))
test_features, test_labels = next(iter(test_dl))

## Practice Model

torch.Size([6, 32, 79, 95, 79])
torch.Size([6, 32, 79, 95, 79])
torch.Size([6, 64, 37, 45, 37])
torch.Size([6, 64, 18, 22, 18])
before drop torch.Size([6, 128, 1, 1, 1])
after drop torch.Size([6, 128])
after lin layer torch.Size([6, 64])
torch.Size([6, 2])
torch.Size([6, 32, 79, 95, 79])
torch.Size([6, 32, 79, 95, 79])
torch.Size([6, 64, 37, 45, 37])
torch.Size([6, 64, 18, 22, 18])
before drop torch.Size([6, 128, 1, 1, 1])
after drop torch.Size([6, 128])
after lin layer torch.Size([6, 64])
torch.Size([6, 2])
torch.Size([6, 32, 79, 95, 79])
torch.Size([6, 32, 79, 95, 79])
torch.Size([6, 64, 37, 45, 37])
torch.Size([6, 64, 18, 22, 18])
before drop torch.Size([6, 128, 1, 1, 1])
after drop torch.Size([6, 128])
after lin layer torch.Size([6, 64])
torch.Size([6, 2])


KeyboardInterrupt: ignored

## Build Model

In [58]:
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    
    #Conv1
    self.conv1 = nn.Conv3d(in_channels = 1, 
                           out_channels = 32, 
                           kernel_size = (1,1,1), 
                           stride = (1,1,1)
                           )
    self.bn1 = nn.BatchNorm3d(32)
    self.conv2 = nn.Conv3d(in_channels = 32, 
                           out_channels = 64, 
                           kernel_size = (7,7,7),
                           stride = (2,2,2)
                           )
    self.bn2 = nn.BatchNorm3d(64)
    self.conv3 = nn.Conv3d(in_channels = 64, 
                           out_channels = 64, 
                           kernel_size = (3,3,3),
                           stride = (2,2,2)
                           )
    self.bn3 = nn.BatchNorm3d(64)
    self.conv4 = nn.Conv3d(in_channels = 64, 
                           out_channels = 128, 
                           kernel_size = (3,3,3),
                           stride = (2,2,2)
                           )
    self.bn4 = nn.BatchNorm3d(128) 
    self.pool1 = nn.AdaptiveAvgPool3d((1,1,1)) #Global Average Pool, takes the average over last two dimensions to flatten 
  
                                                             
    # Fully connected layer
    self.fc1 = nn.Linear(128,64) # need to find out the size where AdaptiveAvgPool 
    self.fc2 = nn.Linear(64, 2) # left with 2 for the two classes                     



  def forward(self, xb):
    xb = self.bn1(F.relu(self.conv1((xb))))
    xb = self.bn2(F.relu(self.conv2((xb)))) # Takes a long time
    xb = self.bn3(F.relu(self.conv3((xb))))
    xb = self.bn4(F.relu(self.conv4((xb))))
    xb = self.pool1(xb)
    xb = xb.view(xb.shape[:2])
    xb = self.fc1(xb)
    xb = self.fc2(xb)
    return xb      

    

In [64]:
# Set to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Get model
model = ConvNet()
model = model.to(device)
print("First model training on GPU")
print(model)

# Initialize other parameters
epochs = 3 #120
learning_rate = 0.0002
loss_func = F.cross_entropy
opt = torch.optim.Adam(model.parameters(), lr = learning_rate)#, momentum = 0.9) #or ADAM/ momentum

First model training on GPU
ConvNet(
  (conv1): Conv3d(1, 32, kernel_size=(1, 1, 1), stride=(1, 1, 1))
  (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv3d(32, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2))
  (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2))
  (bn3): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(2, 2, 2))
  (bn4): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): AdaptiveAvgPool3d(output_size=(1, 1, 1))
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=2, bias=True)
)


In [65]:
# Run Model
for epoch in range(1, 1+epochs):
  model.train()
  i = 1
  print('epoch', epoch)
  for xb, yb in train_dl:
    print('batch', i)
    i = i+1

    xb = xb.float()
    pred = model(xb)
    loss = loss_func(pred, yb)
    print('Loss', loss)

    loss.backward()
    opt.step()
    opt.zero_grad()

  model.eval()

  print(epoch, valid_loss / len(valid_dl))

epoch 1
batch 1
Loss tensor(0.6779, grad_fn=<NllLossBackward0>)
batch 2
Loss tensor(0.6765, grad_fn=<NllLossBackward0>)
batch 3
Loss tensor(0.6590, grad_fn=<NllLossBackward0>)
batch 4
Loss tensor(0.7258, grad_fn=<NllLossBackward0>)
batch 5
Loss tensor(0.6836, grad_fn=<NllLossBackward0>)
batch 6
Loss tensor(0.7156, grad_fn=<NllLossBackward0>)
batch 7
Loss tensor(0.7124, grad_fn=<NllLossBackward0>)
batch 8
Loss tensor(0.7137, grad_fn=<NllLossBackward0>)
batch 9
Loss tensor(0.7218, grad_fn=<NllLossBackward0>)
batch 10
Loss tensor(0.6575, grad_fn=<NllLossBackward0>)
batch 11
Loss tensor(0.7254, grad_fn=<NllLossBackward0>)
batch 12
Loss tensor(0.6792, grad_fn=<NllLossBackward0>)
batch 13
Loss tensor(0.6600, grad_fn=<NllLossBackward0>)
batch 14
Loss tensor(0.7117, grad_fn=<NllLossBackward0>)


NameError: ignored

## Training

In [ ]:
# Set to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Get model
model = ConvNet()
model = model.to(device)
print("First model training on GPU")
print(model)

# Initialize other parameters
epochs = 5 #120
learning_rate = 0.001
criterion = nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)#, momentum = 0.9) #or ADAM/ momentum

fit(epochs, model, )

In [ ]:
def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

In [ ]:
accuracy_stats = {
    'train': [],
    'val': []
  }

print(accuracy_stats)

loss_stats = {
    'train': [],
    'val': []
    }
print(loss_stats)



def train_test_model(epochs, model, opt, train_dl, test_dl, device):
  for epoch in range(1, epochs+1):

    # TRAINING *****************************************************************

    train_epoch_loss = 0
    train_epoch_acc = 0

    # set model in training mode 
    model.train()
    print('\nEpoch$ : %d'%epoch)
    for xb, yb in tqdm(train_dl):
      xb = xb.to(device)
      y_train_batch = y_train_batch.to(device) 

      #print(x_train_batch.shape)

      # sets gradients to 0 to prevent interference with previous epoch
      optimizer.zero_grad()
    
      # Forward pass through NN
      y_pred = model(xb)#.to(float)
      train_loss = criterion(y_pred, yb)
      train_acc = accuracy(y_pred, yb)

      # Backward pass, updating weights
      train_loss.backward()
      opt.step()

      # Statistics
      train_epoch_loss += train_loss.item()
      train_epoch_acc += train_acc.item()


    # VALIDATION****************************************************************   
    
    with torch.set_grad_enabled(False):
      val_epoch_loss = 0
      val_epoch_acc = 0

      model.eval()
      for x_val_batch, y_val_batch in tqdm(validate_loader):
      
        x_val_batch =  x_val_batch.to(device)#.to(float)
        y_val_batch = y_val_batch.to(device)
            
        # Forward pass
        y_val_pred = model(x_val_batch)#.to(float)   
        val_loss = criterion(y_val_pred, y_val_batch)
        val_acc = accuracy(y_val_pred, y_val_batch)
            
        val_epoch_loss += val_loss.item()
        val_epoch_acc += val_acc.item()

    # Prevent plateauing validation loss 
    #scheduler.step(val_epoch_loss/len(validate_loader))

        
    loss_stats['train'].append(train_epoch_loss/len(train_loader))
    loss_stats['val'].append(val_epoch_loss/len(validate_loader))
    accuracy_stats['train'].append(train_epoch_acc/len(train_loader))
    accuracy_stats['val'].append(val_epoch_acc/len(validate_loader))
                              
    
    print(f'Epoch {epoch+0:03}: Train Loss: {train_epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(validate_loader):.5f}') 
    print(f'Train Acc: {train_epoch_acc/len(train_loader):.3f} | Val Acc: {val_epoch_acc/len(validate_loader):.3f}')

      




# def accuracy(y_pred, y_test):
#   # Calculating model accuracy at each epoch 
#   y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
#   _, y_pred_prob = torch.max(y_pred_softmax, dim = 1)
#   correct_pred = (y_pred_prob == y_test).float()
#   acc = correct_pred.sum() / len(correct_pred)
#   acc = torch.round(acc * 100)

#   return acc



     





# if __name__ == '__main__':
#   train_val_model(epochs)

{'train': [], 'val': []}
{'train': [], 'val': []}

Epoch$ : 1


NameError: ignored

In [ ]:
model = ConvNet()

epochs = 5 #120
learning_rate = 0.001
loss_func = F.cross_entropy
opt = torch.optim.Adam(model.parameters(), lr = learning_rate)#, momentum = 0.9) #or ADAM/ momentum

def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)



for epoch in range(epochs):
    model.train()
    for xb, yb in train_dl:
      xb = xb.float()
      loss_batch(model, loss_func, xb, yb, opt)

    model.eval()
    # with torch.no_grad():
    #     losses, nums = zip(
    #         *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
    #     )
    # val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

    # print(epoch, val_loss)